# P.76 ナップサック問題

In [12]:
items = {1,2,3,4,5}
costs = {1:50,2:40,3:10,4:70,5:55}
weights = {1: 7,2: 5,3: 1,4: 9,5: 6}
capacity = 15

In [17]:
# ヒューリスティックの評価値を作成
ratio = {j:c[j]/w[j] for j in items}
sorted_items= [key for key, val in sorted(ratio.items(), key=lambda x:x[1], reverse=True)]

[3, 5, 2, 4, 1]

In [19]:
for j in sorted_items:
    print('c[%d]/w[%d] = %1f' % (j,j,ratio[j]))

c[3]/w[3] = 10.000000
c[5]/w[5] = 9.166667
c[2]/w[2] = 8.000000
c[4]/w[4] = 7.777778
c[1]/w[1] = 7.142857


In [27]:
# ナップサック問題の貪欲法による解法
x = {j:0 for j in sorted_items}
cap =capacity
val = 0

for j in sorted_items:
    if w[j] <= cap:
        cap -= w[j]
        x[j] = 1
        val += c[j]

print('設計変数', x)
print('総価格 = ', val)
print('総重量 = ', capacity-cap)

設計変数 {1: 0, 2: 1, 3: 1, 4: 0, 5: 1}
総価格 =  105
総重量 =  12


In [34]:
def solve_knapsack_greedy(items, costs, weights, capacity, relax=False, verbose=True):
    
    # ヒューリスティックの評価値(評価値/重量)を作成
    ratio = {j:costs[j]/weights[j] for j in items}
    
    # 評価値が大きい順にソート
    sorted_items= [key for key, val in sorted(ratio.items(), key=lambda x:x[1], reverse=True)]
    
    # ナップサック問題の貪欲法による解法
    x = {j:0 for j in sorted_items}
    cap =capacity
    cost = 0
    
    for j in sorted_items:       
        if weights[j] <= cap:
            x[j] = 1
            cap -= weights[j]
            cost += costs[j]
        elif relax and cap > 0:
            # 連続値緩和する場合
            x[j] = cap/weights[j]
            cap = 0
            cost += costs[j]
            break

    if verbose:
        print('設計変数', x)
        print('総価格 = ', cost)
        print('総重量 = ', capacity-cap)
    
    return cost, x

In [35]:
solve_knapsack_greedy(items, costs, weights, capacity, relax=True)

設計変数 {1: 0, 2: 1, 3: 1, 4: 0.3333333333333333, 5: 1}
総価格 =  175
総重量 =  15


(175, {1: 0, 2: 1, 3: 1, 4: 0.3333333333333333, 5: 1})

In [81]:
import numpy as np

class KnapsackProblem(object):
    """
    The definition of KnapSackProblem
    """
    
    
    def __init__(self, name, capacity, items, costs, weights, 
                 zeros=set(), ones=set()):
        self.name = name
        self.capacity = capacity
        self.items = items
        self.costs = costs
        self.weights = weights
        self.zeros = zeros # set()
        self.ones = ones # set()
        self.lb = -100
        self.ub = -100
        
        # ヒューリスティックの評価値(評価値/重量)を作成
        ratio = {j:self.costs[j]/self.weights[j] for j in self.items}
        
        # 評価値が大きい順にソート
        self.sorted_items= [key for key, val in sorted(ratio.items(), 
                                                       key=lambda x:x[1], 
                                                       reverse=True)]
        # 下界・上界の割付結果を保存
        self.xlb = {j:0 for j in self.items}
        self.xub = {j:0 for j in self.items}
        self.bi = None # 値が分数である品物
        
    def get_bounds(self):
        """
        Calculate the upper and lower bounds
        """
        # 決定済みの解の登録
        for j in self.zeros:
            self.xlb[j] = self.xub[j] = 0
        for j in self.ones:
            self.xlb[j] = self.xub[j] = 1 
        
        # 決定済みの解の評価値と総重量を計算
        bweight = np.sum(self.weights[j] for j in self.ones)
        bcost = np.sum(self.costs[j] for j in self.ones)
        
        # 現時点で容量超えの場合、最悪値を設定して終了
        if self.capacity < bweight:
            self.lb = self.ub = -100
            return 0
        
        # 未決定のアイテムを評価値順に取得
        ritems = self.items - self.zeros - self.ones
        #sitems = [j for j in self.sorted_items if j in ritems]
        rcapacity = self.capacity - bweight
    
        # 下界・上界を取得
        rlb, rxlb = self.solve_knapsack_greedy(ritems, self.costs, self.weights, 
                                               capacity=rcapacity, relax=False)
        rub, rxub = self.solve_knapsack_greedy(ritems, self.costs, self.weights, 
                                               capacity=rcapacity, relax=True)
        
        self.lb = bcost + rlb
        self.ub = bcost + rub
        
        for i in ritems:
            self.xlb[i] = rxlb[i]
            self.xub[i] = rxub[i]
            
            if 0 < rxub[i] < 1:
                self.bi = i
    
    def solve_knapsack_greedy(self, items, costs, weights, capacity, relax=False, verbose=False):
        # ヒューリスティックの評価値(評価値/重量)を作成
        ratio = {j:costs[j]/weights[j] for j in items}

        # 評価値が大きい順にソート
        sorted_items= [key for key, val in sorted(ratio.items(), key=lambda x:x[1], reverse=True)]

        # ナップサック問題の貪欲法による解法
        x = {j:0 for j in sorted_items}
        cap =capacity
        cost = 0

        for j in sorted_items:       
            if weights[j] <= cap:
                x[j] = 1
                cap -= weights[j]
                cost += costs[j]
            elif relax and cap > 0:
                # 連続値緩和する場合
                x[j] = cap/weights[j]
                cap = 0
                cost += costs[j]
                break

        if verbose:
            print('設計変数', x)
            print('総価格 = ', cost)
            print('総重量 = ', capacity-cap)

        return cost, x
    
    def __str__(self):
        """
        KnapsackProblemの情報を印字
        """
        s = (
            'Name = '+ self.name + ', Capacity = ' + str(self.capacity) + '\n' + 
            'Items = ' + str(self.items) + '\n' + 
            'Weights = ' + str(self.weights) + '\n' + 
            'zeros = ' + str(self.zeros) + ', ones = ' + str(self.ones) + '\n' + 
            'SortedItems = ' + str(self.sorted_items) + '\n' + 
            'lb = ' + str(self.lb) + '\n' + 
            'ub = ' + str(self.ub) + '\n' + 
            'xlb = ' + str(self.xlb) + '\n' + 
            'xub = ' + str(self.xub) + '\n' + 
            'bi = ' + str(self.bi)
        )
        return s

In [88]:
kp = KnapsackProblem(name='Test', capacity=capacity, 
                     items=items, costs=costs, 
                     weights=weights, ones=set([1]))

In [89]:
kp.get_bounds()

In [90]:
print(kp)

Name = Test, Capacity = 15
Items = {1, 2, 3, 4, 5}
Weights = {1: 7, 2: 5, 3: 1, 4: 9, 5: 6}
zeros = set(), ones = {1}
SortedItems = [3, 5, 2, 4, 1]
lb = 115
ub = 155
xlb = {1: 1, 2: 0, 3: 1, 4: 0, 5: 1}
xub = {1: 1, 2: 0.2, 3: 1, 4: 0, 5: 1}
bi = 2


In [143]:
# 分枝限定法
from collections import deque

def solve_knapsack_branch_and_bound(capacity, items, costs, weights):
    """
    分枝限定法を用いて、ナップサック問題の厳密最適解を出力する
    """
    
    queue = deque()

    # 分枝なしで解いたKPを初期の暫定解とする
    root = KnapsackProblem('KP', capacity=capacity, items=items, costs=costs, weights=weights)
    root.get_bounds()
    best = root

    queue.append(root)

    while len(queue) > 0:
        p = queue.popleft() 
        p.get_bounds()

        # 暫定解を更新する可能性がない場合はcontinue
        if p.ub <= best.lb:
            print('<Bound>:', p.name,'lb =', p.lb,'ub =',p.ub)
            continue

        # 暫定解の下界よりも現在の解の下界のほうが良い場合、暫定解を更新
        if p.lb > best.lb:
            best = p
            print('Best:', best.name,'lb =', best.lb,'ub =',best.ub)

        # 子問題を作って分枝する
        if p.ub > p.lb:
            k = p.bi

            #print('Branch of ' + str(k))
            print('<Branch>:', p.name + ' lb = ' + str(p.lb) + ' ub = '+str(p.ub)
                  + ' >>> ' + p.name + '+'  + str(k))
            
            # k=1の枝
            p1 = KnapsackProblem(name=p.name + '+('  + str(k)+'->1)',
                                 capacity = p.capacity,
                                 items=p.items,
                                 costs=p.costs,
                                 weights=p.weights,
                                 zeros=p.zeros, 
                                 ones=p.ones.union({k}))
            queue.append(p1)

            # k=0の枝
            p2 = KnapsackProblem(name=p.name + '+('  + str(k)+'->0)',
                                 capacity = p.capacity,
                                 items=p.items,
                                 costs=p.costs,
                                 weights=p.weights,
                                 zeros=p.zeros.union({k}), 
                                 ones=p.ones)
            queue.append(p2)

    if best.lb == best.ub:
        status = 'Optimal'
    else:
        status = 'Not Solved'
    #print(status, best.lb, best.xlb)
    return [status, best.lb, best.xlb]

In [144]:
solve_knapsack_branch_and_bound(capacity, items, costs, weights)

<Branch>: KP lb = 105 ub = 175 >>> KP+4
Best: KP+(4->1) lb = 120 ub = 135
<Branch>: KP+(4->1) lb = 120 ub = 135 >>> KP+(4->1)+5
<Branch>: KP+(4->0) lb = 105 ub = 155 >>> KP+(4->0)+1
Best: KP+(4->1)+(5->1) lb = 125 ub = 125
<Bound>: KP+(4->1)+(5->0) lb = 120 ub = 120
<Branch>: KP+(4->0)+(1->1) lb = 115 ub = 155 >>> KP+(4->0)+(1->1)+2
<Bound>: KP+(4->0)+(1->0) lb = 105 ub = 105
<Branch>: KP+(4->0)+(1->1)+(2->1) lb = 100 ub = 155 >>> KP+(4->0)+(1->1)+(2->1)+5
<Bound>: KP+(4->0)+(1->1)+(2->0) lb = 115 ub = 115
<Bound>: KP+(4->0)+(1->1)+(2->1)+(5->1) lb = -100 ub = -100
<Bound>: KP+(4->0)+(1->1)+(2->1)+(5->0) lb = 100 ub = 100


['Optimal', 125, {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}]

# P.89 列生成法

列生成法(を双対問題の側から見たもの)の枠組み
- Step A 少なくとも実行可能解は得られる程度の制約式を用意し，それで暫定的に線形計画問題を作る.
- Step B その暫定的線形計画問題の最適解を得る. 
- Step C 暫定的問題の最適解を入力定数とみなして，その最適解が満たしていない制約式を探索する. 
    - Step C-1 そのような制約式がないならば，手続きを終了する.
    - Step C-2 そうでないならば，その制約式を暫定的線形計画問題に加え Step B へ移る
    
http://www.orsj.or.jp/archive2/or57-04/or57_4_198.pdf

注釈<br>
この記事では，列生成法の設計と実行をわかりやすく説明するために，<br>
双対問題の観点から，制約式を増やす手法であると説明した.<br>
しかし，双対問題は主問題と一対一に対応するので，実は双対問題を経由する必要はない.<br>
先ほどの手続きを，主問題の観点から書き直すと，変数を生成していることになる.<br>
線形計画問題を行列形式で記述すると，変数の生成は行列の列生成になる.<br>
これが列生成法とよばれる所以であろう.<br>

In [145]:
from pulp import * 
import numpy as np
MEPS = 10e-8

In [148]:
def KPS(capacity, items, costs, weights):
    """
    KnapsackProblemをPuLPで解く
    """
    knapsack = LpProblem(name='kanpsack', sense=LpMaximize)
    # アイテム個数分の設計変数を生成
    x = {j:LpVariable('x'+str(j), lowBound=0, cat=LpBinary) for j in items}
    # 目的関数
    knapsack += lpSum(costs[j]*x[j] for j in items)
    # 制約条件
    knapsack += lpSum(weights[j]*x[j] for j in items) <= capacity, 'weights'
    
    status = LpStatus[knapsack.solve()]
    # 最適解を取得
    obj_opt = value(knapsack.objective)
    var_opt = {j: int(x[j].varValue) for j in items}
    
    return status, obj_opt, var_opt

In [149]:
KPS(capacity, items, costs, weights)

('Optimal', 125.0, {1: 0, 2: 0, 3: 0, 4: 1, 5: 1})

In [238]:
# ビンパッキング問題の生成
def make_binpacking_weights(n_item):

    items = set(range(n_item)) # 10
    np.random.seed(1)

    w = {i:np.random.randint(5,10) for i in items}
    w2 = [w[i] for i in items]

    print(w2)
    
    return w

In [239]:
capacity = 25
w = make_binpacking_weights(n_item=10)

[8, 9, 5, 6, 8, 5, 5, 6, 9, 9]


In [240]:
def binpacking(capacity, w):
    """
    列生成法を用いて、大規模なビンパッキング問題を解く
    """
    # a_iJ:アイテムiがアイテム集合Jに含まれている場合1,そうでない場合0
    # 双対問題を用いて、必要なビン(アイテムの組合せ)を生成
    A, obj_opt_dual = solve_dual_binpacking(capacity, w)

    # 元問題を、列生成法で得られたアイテムの組合せを用いて解く
    m,n = A.shape
    primal = LpProblem(name='P(K)', sense=LpMinimize)

    # x_j = 1, アイテム組合せjを使う場合1
    x = [LpVariable('x'+str(j), lowBound=0, cat='Binary') for j in range(n)]

    # 目的関数(使うアイテムの組合せ数の最小化)
    primal += lpSum(x[j] for j in range(n))
    # 制約条件
    for i in range(m):
        primal += lpDot(A[i], x) >= 1, 'ineq'+str(i)

    primal.solve()

    if value(primal.objective) - obj_opt_dual < 1.0 - MEPS:
        print('Optimal solution found: ')
    else:
        print('Approximated solution found: ')
    
    
    # 結果集計
    K = [j for j in range(n) if x[j].varValue > MEPS]

    result = []
    result_dict = []

    itms =set(range(m))
    for j in K:
        J = {i for i in range(m) if A[i,j] > MEPS and i in itms}
        r = [w[i] for i in J]
        #r = {i:w[i] for i in J}
        itms -= J
        result.append(J)
    
    print('Status =', LpStatus[primal.status])
    print('Objective =', value(primal.objective))
    print('Item allocation =', )
    for i,alloc in enumerate(result):     
        print('i =', i+1, ':', alloc)

In [241]:
def solve_dual_binpacking(capacity, w, verbose=False):
    """
    ビンパッキング問題の双対問題を列生成法を用いて解き、
    得られた制約条件Aと最適解の目的関数値を返す
    """

    m = len(w)
    items = set(range(m))

    # a_iJ:アイテムiがアイテム集合Jに含まれている場合1,そうでない場合0
    # ビンパッキング問題を解くために最低限必要な制約条件式を生成
    A = np. identity(m)

    # 双対問題の部分問題D(K)
    solved = False
    columns = 0
    dual = LpProblem(name='D(K)', sense=LpMaximize)
    y = [LpVariable('y'+str(i), lowBound=0) for i in items]

    # 目的関数
    dual += lpSum(y[i] for i in items)
    # 制約条件
    for j in range(len(A.T)):
        # アイテム集合Jに、各アイテムを1つだけ詰めた集合を作成
        # これが「最適限、ビンパッキング問題が解けるだけの制約式」になる
        dual += lpDot(A.T[j], y) <= 1, 'ineq'+str(j)

    while not solved:
        # 現時点のアイテム集合だけを使うとして
        # ビンパッキング問題の双対問題を部分問題としてを解く
        dual.solve()

        # 双対問題の部分問題D(K)の最適解y*(K)が双対問題Dの最適解y*であるか判定
        # すなわち、「元問題のすべての制約式が満たすべき条件を満たし(=制約条件として成立しうる)、
        # かつ、現時点の最適解y*(K)を入力すると、双対問題の制約式条件を満たさない制約条件」を見つける。
        # そのような制約式が存在しないことがわかれば、現時点の最適解y*(K)は双対問題Dの最適解y*と同じ
        costs = {i:y[i].varValue for i in items}
        weights = {i:w[i] for i in items}
        
        if verbose:
            print('Columns =', columns, end=', ')

        # 今回の問題はたまたま「最悪の制約条件式を見つける=ナップサック問題を解く」のため
        # ナップサック問題を解いて、目的関数値が1以上の答えが見つかった場合、
        # その解solが新たに追加するべき制約条件式a_iJに相当する
        status, val, sol = KPS(capacity, items, costs, weights)

        # 制約条件を満たさない
        if val >= 1.0 + MEPS:
            a = np.array([int(sol[i]) for i in items])
            if verbose:
                print('Generated a =', a)
            
            dual += lpDot(a,y) <= 1, 'ineq'+str(m+columns)
            A = np.hstack((A, a.reshape((-1,1))))
            columns += 1
        else:
            if verbose:
                print('Solved!')
            solved = True

    print('Generated columns =', columns)
    
    return A, value(dual.objective)

In [242]:
%%time
binpacking(capacity, w)

Generated columns = 20
Optimal solution found: 
Status = Optimal
Objective = 3.0
Item allocation =
i = 1 : {0, 1, 4}
i = 2 : {8, 3, 5, 6}
i = 3 : {9, 2, 7}
CPU times: user 95.9 ms, sys: 136 ms, total: 232 ms
Wall time: 566 ms


In [243]:
# 別の定式化によるビンパッキング問題の最適化
# しかし、アイテム数が多くなる(>100)と
# 計算が終わらなくなるので、列生成法のような工夫が必要
def binpacking2(capacity, w):
    n = len(w)
    items =range(n)

    bpprob = LpProblem(name='Binpacking2', sense=LpMinimize)
    # ビンjを使うとき1
    z = [LpVariable('z'+str(j), lowBound=0, cat=LpBinary) for j in items]
    # x_ij:アイテムiをビンjに詰めるとき1
    x = [[LpVariable('x'+str(i)+'_'+str(j), lowBound=0, cat=LpBinary) for j in items] for i in items]

    # 目的関数(使うビン数の最小化)
    bpprob += lpSum(z[i] for i in items)
    # 制約条件
    for i in items:
        # すべてのアイテムはどれかのビンに入っている
        bpprob += lpSum(x[i][j] for j in items) == 1
    for j in items:
        # 利用するすべてのビンについて、ビン内のアイテムの合計容量がcapacity以下
        bpprob += lpSum(x[i][j]*w[i] for i in items) <= capacity*z[j]

    bpprob.solve()

    # 結果集計
    result = []
    for j in items:
        if z[j].varValue > MEPS:
            # r = {i:w[i] for i in items if x[i][j].varValue > MEPS}
            r = {i for i in items if x[i][j].varValue > MEPS}
            result.append(r)
    
    print('Status =', LpStatus[bpprob.status])
    print('Objective =', value(bpprob.objective))
    print('Item allocation =', )
    for i,alloc in enumerate(result):     
        print('i =', i+1, ':', alloc)

In [244]:
%%time
binpacking2(capacity, w)

Status = Optimal
Objective = 3.0
Item allocation =
i = 1 : {2, 3, 6, 7}
i = 2 : {8, 9, 5}
i = 3 : {0, 1, 4}
CPU times: user 9.81 ms, sys: 8.03 ms, total: 17.8 ms
Wall time: 36.8 ms


# 列生成法による計算時間の違い

In [250]:
import time

capacity = 25
n_items = [5, 10, 15, 20, 25, 30, 35, 40] # ,45, 50]

e_times_1 = []
e_times_2 = []

for i, n_item in enumerate(n_items):
    
    w_i = make_binpacking_weights(n_item)
    print ("i=", i+1, "n_item=", n_item)
    print ("")
    
    start_1 = time.time()
    binpacking(capacity, w_i)
    e_time_1 = time.time() - start_1
    e_times_1.append(e_time_1)
    print ("")
    
    start_2 = time.time()
    binpacking2(capacity, w_i)
    e_time_2 = time.time() - start_2
    e_times_2.append(e_time_2)
    print ("")
    
    print ("e_time_1:{0}".format(e_time_1) + "[s]")
    print ("e_time_2:{0}".format(e_time_2) + "[s]")
    print ("/////////")
    print ("")

[8, 9, 5, 6, 8]
i= 1 n_item= 5

Generated columns = 9
Optimal solution found: 
Status = Optimal
Objective = 2.0
Item allocation =
i = 1 : {1, 3, 4}
i = 2 : {0, 2}

Status = Optimal
Objective = 2.0
Item allocation =
i = 1 : {1, 2, 4}
i = 2 : {0, 3}

e_time_1:0.24814510345458984[s]
e_time_2:0.015064001083374023[s]
/////////

[8, 9, 5, 6, 8, 5, 5, 6, 9, 9]
i= 2 n_item= 10

Generated columns = 20
Optimal solution found: 
Status = Optimal
Objective = 3.0
Item allocation =
i = 1 : {0, 1, 4}
i = 2 : {8, 3, 5, 6}
i = 3 : {9, 2, 7}

Status = Optimal
Objective = 3.0
Item allocation =
i = 1 : {2, 3, 6, 7}
i = 2 : {8, 9, 5}
i = 3 : {0, 1, 4}

e_time_1:0.5982820987701416[s]
e_time_2:0.040142059326171875[s]
/////////

[8, 9, 5, 6, 8, 5, 5, 6, 9, 9, 6, 7, 9, 7, 9]
i= 3 n_item= 15

Generated columns = 31
Optimal solution found: 
Status = Optimal
Objective = 5.0
Item allocation =
i = 1 : {2, 5, 6, 7}
i = 2 : {11, 10, 3}
i = 3 : {0, 9, 4}
i = 4 : {12, 13, 14}
i = 5 : {8, 1}

Status = Optimal
Objective =

KeyboardInterrupt: 